In [ ]:
import tensorflow as tf
tf.__version__

'2.6.0'

# 필요한 도구 임포트



In [ ]:
import nltk
import numpy as np
import re
import shutil
import tensorflow as tf
from tensorflow.keras.layers import Embedding, GRU, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd
import os
import unicodedata
import urllib3
import zipfile
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import time

In [ ]:
import pandas as pd
import urllib3
import zipfile
import shutil
import os
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

# 데이터 로드

In [ ]:
http = urllib3.PoolManager()
url ='http://www.manythings.org/anki/fra-eng.zip'
filename = 'fra-eng.zip'
path = os.getcwd()
zipfilename = os.path.join(path, filename)
with http.request('GET', url, preload_content=False) as r, open(zipfilename, 'wb') as out_file:       
    shutil.copyfileobj(r, out_file)

with zipfile.ZipFile(zipfilename, 'r') as zip_ref:
    zip_ref.extractall(path)

In [ ]:
lines= pd.read_csv('fra.txt', names=['src', 'tar', 'lic'], sep='\t')
del lines['lic']
len(lines)

178009

In [ ]:
lines

,src,tar
0,Go.,Va !
1,Hi.,Salut !
2,Hi.,Salut.
3,Run!,Cours !
4,Run!,Courez !
...,...,...
178004,"Top-down economics never works, said Obama. ""T...","« L'économie en partant du haut vers le bas, ç..."
178005,A carbon footprint is the amount of carbon dio...,Une empreinte carbone est la somme de pollutio...
178006,Death is something that we're often discourage...,La mort est une chose qu'on nous décourage sou...
178007,Since there are usually multiple websites on a...,Puisqu'il y a de multiples sites web sur chaqu...


총 33,000개의 샘플을 사용할 예정입니다. 이 값을 변수에 지정합니다

In [ ]:
num_samples = 33000

# 전처리 함수

In [ ]:
def unicode_to_ascii(s):
  return ''.join(c for c in unicodedata.normalize('NFD', s)
      if unicodedata.category(c) != 'Mn')

In [ ]:
def preprocess_sentence(sent):
    sent = unicode_to_ascii(sent.lower())
    sent = re.sub(r"([?.!,¿])", r" \1", sent)
    sent = re.sub(r"[^a-zA-Z!.?]+", r" ", sent)
    sent = re.sub(r"\s+", " ", sent)
    return sent

In [ ]:
def load_preprocessed_data():
    encoder_input, decoder_input, decoder_target = [], [], []

    with open("fra.txt", "r") as lines:
        for i, line in enumerate(lines):

            # source 데이터와 target 데이터 분리
            src_line, tar_line, _ = line.strip().split('\t')

            # source 데이터 전처리
            src_line = [w for w in preprocess_sentence(src_line).split()]

            # target 데이터 전처리
            tar_line = preprocess_sentence(tar_line)
            tar_line_in = [w for w in ("<sos> " + tar_line).split()]
            tar_line_out = [w for w in (tar_line + " <eos>").split()]

            encoder_input.append(src_line)
            decoder_input.append(tar_line_in)
            decoder_target.append(tar_line_out)

            if i == num_samples - 1:
                break
                
    return encoder_input, decoder_input, decoder_target

In [ ]:
# 인코딩 테스트
en_sent = u"Have you had dinner?"
fr_sent = u"Avez-vous déjà diné ?"
print(preprocess_sentence(en_sent))
print(preprocess_sentence(fr_sent).encode('utf-8'))

have you had dinner ?
b'avez vous deja dine ?'


In [ ]:
sents_en_in, sents_fra_in, sents_fra_out = load_preprocessed_data()

# 인코더의 입력과 디코더의 입, 출력 구성

In [ ]:
print(sents_en_in[:5])
print(sents_fra_in[:5])
print(sents_fra_out[:5])

[['go', '.'], ['hi', '.'], ['hi', '.'], ['run', '!'], ['run', '!']]
[['<sos>', 'va', '!'], ['<sos>', 'salut', '!'], ['<sos>', 'salut', '.'], ['<sos>', 'cours', '!'], ['<sos>', 'courez', '!']]
[['va', '!', '<eos>'], ['salut', '!', '<eos>'], ['salut', '.', '<eos>'], ['cours', '!', '<eos>'], ['courez', '!', '<eos>']]


이제 케라스 토크나이저를 통해 단어 집합을 생성하고, 텍스트 시퀀스를 정수 시퀀스로 변환하는 정수 인코딩 과정을 거침.

In [ ]:
tokenizer_en = Tokenizer(filters="", lower=False)
tokenizer_en.fit_on_texts(sents_en_in)
encoder_input = tokenizer_en.texts_to_sequences(sents_en_in)
encoder_input = pad_sequences(encoder_input, padding="post")

tokenizer_fra = Tokenizer(filters="", lower=False)
tokenizer_fra.fit_on_texts(sents_fra_in)
tokenizer_fra.fit_on_texts(sents_fra_out)

decoder_input = tokenizer_fra.texts_to_sequences(sents_fra_in)
decoder_input = pad_sequences(decoder_input, padding="post")

decoder_target = tokenizer_fra.texts_to_sequences(sents_fra_out)
decoder_target = pad_sequences(decoder_target, padding="post")

샘플은 총 33,000개 존재하며 영어 문장의 길이는 8, 프랑스어 문장의 길이는 16. 단어 집합의 크기를 정의.



In [ ]:
print(encoder_input.shape)
print(decoder_input.shape)
print(decoder_target.shape)

(33000, 8)
(33000, 16)
(33000, 16)


In [ ]:
vocab_size_en = len(tokenizer_en.word_index) + 1
vocab_size_fra = len(tokenizer_fra.word_index) + 1
print("영어 단어 집합의 크기 (en): {:d}, 프랑스어 단어 집합의 크기 (spa): {:d}".format(vocab_size_en, vocab_size_fra))

영어 단어 집합의 크기 (en): 4663, 프랑스어 단어 집합의 크기 (spa): 8038


단어 집합의 크기는 각각 4,647개와 8,022개. 단어로부터 정수를 얻는 딕셔너리와 정수로부터 단어를 얻는 딕셔너리를 각각 만들어 줌. 이들은 훈련을 마치고 예측 과정과 실제값과 결과를 비교하는 경우에 사용됨.

In [ ]:
src_to_index = tokenizer_en.word_index
index_to_src = tokenizer_en.index_word

tar_to_index = tokenizer_fra.word_index
index_to_tar = tokenizer_fra.index_word

이제 테스트 데이터를 분리할 차례. 테스트 데이터를 분리하기 전에, 적절한 분포를 갖도록 데이터를 섞어주는 과정을 진행. 이를 위해서 우선 순서가 섞인 정수 시퀀스 리스트를 만듬.

In [ ]:
max_src_len = encoder_input.shape[1]
max_tar_len = decoder_input.shape[1]

In [ ]:
src_vocab_size = len(tokenizer_en.word_index) + 1
tar_vocab_size = len(tokenizer_fra.word_index) + 1

In [ ]:
print(max_src_len)
print(max_tar_len)

8
16


In [ ]:
indices = np.arange(encoder_input.shape[0])
np.random.shuffle(indices)
print(indices)

[ 7619 26228 14062 ... 27462 27058 22843]


In [ ]:
encoder_input = encoder_input[indices]
decoder_input = decoder_input[indices]
decoder_target = decoder_target[indices]

임의로 30,997번째 샘플을 출력. 이때, decoder_input과 decoder_target은 데이터의 구조상으로 앞에 붙은 <sos> 토큰과 뒤에 붙은 <eos>을 제외하면 동일한 정수 시퀀스를 가져야하므로 이를 확인해주면 됨.

In [ ]:
encoder_input[30997]

array([ 25, 354,   8,   6, 833,   1,   0,   0], dtype=int32)

In [ ]:
decoder_input[30997]

array([   2,   71,  261,    5, 2283,    1,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0], dtype=int32)

In [ ]:
decoder_target[30997]

array([  71,  261,    5, 2283,    1,    3,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0], dtype=int32)

33,000개의 10%에 해당되는 3,300개의 데이터를 테스트 데이터로 사용.



In [ ]:
n_of_val = int(33000*0.1)
print(n_of_val)

3300


In [ ]:
encoder_input_train = encoder_input[:-n_of_val]
decoder_input_train = decoder_input[:-n_of_val]
decoder_target_train = decoder_target[:-n_of_val]

encoder_input_test = encoder_input[-n_of_val:]
decoder_input_test = decoder_input[-n_of_val:]
decoder_target_test = decoder_target[-n_of_val:]

훈련 데이터와 테스트 데이터의 크기(shape)를 출력해 봄.



In [ ]:
print(encoder_input_train.shape)
print(decoder_input_train.shape)
print(decoder_target_train.shape)
print(encoder_input_test.shape)
print(decoder_input_test.shape)
print(decoder_target_test.shape)

# 모델 설계

In [ ]:
latent_dim = 50

In [ ]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Masking
from tensorflow.keras.models import Model

인코더를 설계함. Masking은 패딩 토큰인 숫자 0의 경우에는 연산을 제외하는 역할을 수행함.



In [ ]:
# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(src_vocab_size, latent_dim)(encoder_inputs)
enc_masking = Masking(mask_value=0.0)(enc_emb)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_masking)
encoder_states = [state_h, state_c]

이제 디코더를 설계함.



In [ ]:
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(tar_vocab_size, latent_dim)
dec_emb = dec_emb_layer(decoder_inputs)
dec_masking = Masking(mask_value=0.0)(dec_emb)

decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_masking,
                                     initial_state=encoder_states)

decoder_dense = Dense(tar_vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

seq2seq의 디코더는 기본적으로 각각의 시점(timestep)에 대해서 다중 클래스 분류 문제를 풀고있음. 매 시점마다 프랑스어 단어 집합의 크기의 선택지에서 단어를 1개 선택하여 이를 이번 시점에서 예측한 단어로 택함. 다중 클래스 분류 문제이므로 위의 설계에서 출력층으로 소프트맥스 함수를 사용함. 이 경우 손실 함수를 지금까지 categorical_crossentropy를 사용해옴.

categorical_crossentropy를 사용하려면 레이블은 원-핫 인코딩이 된 상태여야 함. 그런데 현재 decoder_outputs의 경우에는 원-핫 인코딩을 하지 않은 상태. 원-핫 인코딩을 하지 않은 상태로, 정수 레이블에 대해서 다중 클래스 분류 문제를 풀고자 하는 경우에는 categorical_crossentropy함수가 아니라 sparse_categorical_crossentropy를 사용하면 됨. 이는 케라스에서 규정한 약속.

In [ ]:
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['acc'])

In [ ]:
model.summary()

NameError: ignored

128개의 배치 크기로 총 50 에포크 학습. 테스트 데이터를 검증 데이터로 사용하여 훈련이 제대로 되고있는지 모니터링.

In [ ]:
model.fit(x=[encoder_input_train, decoder_input_train], y=decoder_target_train, \
          validation_data = ([encoder_input_test, decoder_input_test], decoder_target_test),
          batch_size=128, epochs=50)

Epoch 1/50
233/233 [==============================] - 13s 56ms/step - loss: 3.1782 - acc: 0.6067 - val_loss: 1.9140 - val_acc: 0.6834
Epoch 2/50
233/233 [==============================] - 11s 45ms/step - loss: 1.7352 - acc: 0.7244 - val_loss: 1.6502 - val_acc: 0.7357
Epoch 3/50
233/233 [==============================] - 11s 45ms/step - loss: 1.5542 - acc: 0.7470 - val_loss: 1.5380 - val_acc: 0.7545
Epoch 4/50
233/233 [==============================] - 11s 45ms/step - loss: 1.4470 - acc: 0.7631 - val_loss: 1.4326 - val_acc: 0.7710
Epoch 5/50
233/233 [==============================] - 11s 45ms/step - loss: 1.3605 - acc: 0.7778 - val_loss: 1.3632 - val_acc: 0.7812
Epoch 6/50
233/233 [==============================] - 11s 45ms/step - loss: 1.2941 - acc: 0.7890 - val_loss: 1.3036 - val_acc: 0.7934
Epoch 7/50
233/233 [==============================] - 11s 45ms/step - loss: 1.2403 - acc: 0.7976 - val_loss: 1.2840 - val_acc: 0.7979
Epoch 8/50
233/233 [==============================] - 11s 45ms

# 테스트 과정을 위한 모델 재구성

seq2seq는 훈련 과정과 테스트 과정에서의 동작 방식이 다름. 그래서 테스트 과정을 위해 모델을 다시 설계해주어야 함. 특히 디코더를 많이 수정해야 함. 우선 테스트 과정에서의 인코더 모델을 설계함.

In [ ]:
# Encode the input sequence to get the "thought vectors"
encoder_model = Model(encoder_inputs, encoder_states)

# 디코더
# 이전 시점의 상태를 보관할 텐서
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs) # 훈련 때 사용했던 임베딩 층을 재사용

# 다음 단어 예측을 위해 이전 시점의 상태를 현 시점의 초기 상태로 사용
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2) # 모든 시점에 대해서 단어 예측


decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

테스트 과정을 위한 모델 설계를 완료. 이제 테스트 과정에서의 동작을 위한 decode_sequence 함수를 구현.



In [ ]:
def decode_sequence(input_seq):
    # 입력으로부터 인코더의 상태를 얻음
    states_value = encoder_model.predict(input_seq)

    # <SOS>에 해당하는 정수 생성
    target_seq = np.zeros((1,1))
    target_seq[0, 0] = tar_to_index['<sos>']

    stop_condition = False
    decoded_sentence = ''

    # stop_condition이 True가 될 때까지 루프 반복
    # 구현의 간소화를 위해서 이 함수는 배치 크기를 1로 가정.
    while not stop_condition:
        # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # 예측 결과를 단어로 변환
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = index_to_tar[sampled_token_index]

         # 현재 시점의 예측 단어를 예측 문장에 추가
        decoded_sentence += ' '+sampled_char

        # <eos>에 도달하거나 정해진 길이를 넘으면 중단.
        if (sampled_char == '<eos>' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
        states_value = [h, c]

    return decoded_sentence

In [ ]:
# 원문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq2src(input_seq):
    temp=''
    for i in input_seq:
        if(i!=0):
            temp = temp + index_to_src[i]+' '
    return temp

# 번역문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq2tar(input_seq):
    temp=''
    for i in input_seq:
        if((i!=0 and i!=tar_to_index['<sos>']) and i!=tar_to_index['<eos>']):
            temp = temp + index_to_tar[i] + ' '
    return temp

In [ ]:
for seq_index in [3,50,100,300,1001]:
  input_seq = encoder_input_train[seq_index: seq_index + 1]
  decoded_sentence = decode_sequence(input_seq)
  
  print("원문 : ",seq2src(encoder_input_train[seq_index]))
  print("번역문 :",seq2tar(decoder_input_train[seq_index]))
  print("예측문 :",decoded_sentence[:-5])
  print("\n")

원문 :  i mopped the floor . 
번역문 : j ai lave par terre . 
예측문 :  j ai ete un fois . 


원문 :  he dislikes me . 
번역문 : il eprouve de l antipathie a mon egard . 
예측문 :  il m a de l dois . 


원문 :  i hate violence . 
번역문 : je deteste la violence . 
예측문 :  je deteste la bonne . 


원문 :  please stop . 
번역문 : veuillez arreter . 
예측문 :  s il te plait ne plait pas ! 


원문 :  they calmed down . 
번역문 : elles se sont calmees . 
예측문 :  ils se sont pas . 




In [ ]:
for seq_index in [3,50,100,300,1001]:
  input_seq = encoder_input_test[seq_index: seq_index + 1]
  decoded_sentence = decode_sequence(input_seq)
  
  print("원문 : ",seq2src(encoder_input_test[seq_index]))
  print("번역문 :",seq2tar(decoder_input_test[seq_index]))
  print("예측문 :",decoded_sentence[:-5])
  print("\n")

원문 :  make an appointment . 
번역문 : prenez rendez vous . 
예측문 :  un importe ! 


원문 :  tom did it all . 
번역문 : tom a tout fait . 
예측문 :  tom fait tout ce coup de de nous . 


원문 :  i m afraid not . 
번역문 : je crains que non . 
예측문 :  je n ai pas de enfants . 


원문 :  is anyone hurt ? 
번역문 : quelqu un est il blesse ? 
예측문 :  quiconque est il blesse ? 


원문 :  i left you a note . 
번역문 : je vous laissai une note . 
예측문 :  je vous ai vu . 


